# 1 SETUP
***

In [4]:
# Data Analysis
import pandas as pd
import numpy as np

# DataViz 
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical

# Notebook Config 
import sys
import os
proj_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(proj_root)

# My Functions
from src.utils import df_summary

In [5]:
df_trans = pd.read_csv('../data/raw/train_transaction.csv')
df_id = pd.read_csv('../data/raw/train_identity.csv')

In [24]:
print(df_trans.head())
print(df_id.head())

   TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   

   card2  card3       card4  card5  ... V330  V331  V332  V333  V334 V335  \
0    NaN  150.0    discover  142.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
1  404.0  150.0  mastercard  102.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
2  490.0  150.0        visa  166.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
3  567.0  150.0  mastercard  117.0  ...  NaN   NaN   NaN   NaN   NaN  NaN   
4  514.0  150.0  mastercard  102.0  ...  0.0   0.0   0.0   0.0   0.0  0.0   

  V336  V337  V338  V339  
0  NaN   NaN   NaN   NaN  
1  NaN   NaN   NaN  

In [6]:
print(f'Transactions: {df_trans.shape}')
print(f'Identity: {df_id.shape}')

Transactions: (590540, 394)
Identity: (144233, 41)


In [7]:
df_trans_summary = df_summary(df_trans)
df_trans_summary[df_trans_summary['Data Null (P%)'] < 80]

,Data Name,Data Type,Data Null (A),Data Null (P%)
294,V240,float64,460110,77.913435
289,V235,float64,460110,77.913435
306,V252,float64,460110,77.913435
303,V249,float64,460110,77.913435
302,V248,float64,460110,77.913435
...,...,...,...,...
17,C1,float64,0,0.000000
18,C2,float64,0,0.000000
30,C14,float64,0,0.000000
1,isFraud,int64,0,0.000000


In [8]:
df_id_summary = df_summary(df_id)
df_id_summary[df_id_summary['Data Null (P%)'] < 80]

,Data Name,Data Type,Data Null (A),Data Null (P%)
18,id_18,float64,99120,68.722137
3,id_03,float64,77909,54.016071
4,id_04,float64,77909,54.016071
33,id_33,object,70944,49.187079
9,id_09,float64,69307,48.052110
10,id_10,float64,69307,48.052110
30,id_30,object,66668,46.222432
32,id_32,float64,66647,46.207872
34,id_34,object,66428,46.056034
14,id_14,float64,64189,44.503685
